<a href="https://colab.research.google.com/github/CM-007/DS_ASS/blob/main/Ass15_Random_Forests(Fraudcheck).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries**

In [36]:
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Random Forest
from sklearn.ensemble import RandomForestClassifier

# Bagging
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Boosting
from sklearn.ensemble import AdaBoostClassifier

# Stacking
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

import warnings
warnings.filterwarnings("ignore")

# **Read Data Set using pandas**

In [2]:
fraud = pd.read_csv("Fraud_check.csv")

# **EDA**

In [3]:
fraud = fraud.rename({'Marital.Status':'Marital_Status',
                      'Taxable.Income':'Taxable_Income',
                      'City.Population':'City_Population',
                      'Work.Experience':'Work_Experience'
                      },axis=1)

In [4]:
fraud.head(10)

,Undergrad,Marital_Status,Taxable_Income,City_Population,Work_Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
5,NO,Divorced,33329,116382,0,NO
6,NO,Divorced,83357,80890,8,YES
7,YES,Single,62774,131253,3,YES
8,NO,Single,83519,102481,12,YES
9,YES,Divorced,98152,155482,4,YES


In [5]:
fraud.shape

(600, 6)

In [6]:
fraud.describe()

,Taxable_Income,City_Population,Work_Experience
count,600.000000,600.000000,600.000000
mean,55208.375000,108747.368333,15.558333
std,26204.827597,49850.075134,8.842147
min,10003.000000,25779.000000,0.000000
25%,32871.500000,66966.750000,8.000000
50%,55074.500000,106493.500000,15.000000
75%,78611.750000,150114.250000,24.000000
max,99619.000000,199778.000000,30.000000


In [7]:
fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital_Status   600 non-null    object
 2   Taxable_Income   600 non-null    int64 
 3   City_Population  600 non-null    int64 
 4   Work_Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


### **Find and Remove NA values**

In [8]:
fraud.isna().sum()

Undergrad          0
Marital_Status     0
Taxable_Income     0
City_Population    0
Work_Experience    0
Urban              0
dtype: int64

# **Changing the categorical variables into dummies**

In [9]:
fraud1 = pd.get_dummies(fraud)

In [10]:
fraud1.head()

,Taxable_Income,City_Population,Work_Experience,Undergrad_NO,Undergrad_YES,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Urban_NO,Urban_YES
0,68833,50047,10,1,0,0,0,1,0,1
1,33700,134075,18,0,1,1,0,0,0,1
2,36925,160205,30,1,0,0,1,0,0,1
3,50190,193264,15,0,1,0,0,1,0,1
4,81002,27533,28,1,0,0,1,0,1,0


## **Set and Add new column as per problem statment**

In [11]:
fraud1['Status'] = fraud1['Taxable_Income'].apply(lambda Income: 'Risky' if Income <= 30000 else 'Good')

In [12]:
fraud1.head()

,Taxable_Income,City_Population,Work_Experience,Undergrad_NO,Undergrad_YES,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Urban_NO,Urban_YES,Status
0,68833,50047,10,1,0,0,0,1,0,1,Good
1,33700,134075,18,0,1,1,0,0,0,1,Good
2,36925,160205,30,1,0,0,1,0,0,1,Good
3,50190,193264,15,0,1,0,0,1,0,1,Good
4,81002,27533,28,1,0,0,1,0,1,0,Good


### **Drop 	Taxable_Income column**

In [13]:
fraud1.drop(['Taxable_Income'], axis=1, inplace=True)

In [14]:
fraud1.head(2)

,City_Population,Work_Experience,Undergrad_NO,Undergrad_YES,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Urban_NO,Urban_YES,Status
0,50047,10,1,0,0,0,1,0,1,Good
1,134075,18,0,1,1,0,0,0,1,Good


### **Value check for y column**

In [15]:
fraud1['Status'].value_counts()

Good     476
Risky    124
Name: Status, dtype: int64

In [16]:
fraud1.columns

Index(['City_Population', 'Work_Experience', 'Undergrad_NO', 'Undergrad_YES',
       'Marital_Status_Divorced', 'Marital_Status_Married',
       'Marital_Status_Single', 'Urban_NO', 'Urban_YES', 'Status'],
      dtype='object')

In [17]:
array = fraud1.values
X = array[:,0:9]
Y = array[:,9]

In [18]:
num_trees = 100
max_features = 3 #max_features
seed = 7 #random_state

In [19]:
kfold = KFold(n_splits=10, random_state=7, shuffle = True)

# **Random Forest Classifier**

In [20]:
#  Random Forest Model Build
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)

In [21]:
results = cross_val_score(model, X, Y, cv=kfold)

In [23]:
print("Random Forest Mode :",round(results.mean()*100,2))

Random Forest Mode : 73.0


# **Ensemble techniques**

## **1.Bagging**

In [25]:
cart = DecisionTreeClassifier()

In [26]:
model_Bagging = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)

In [34]:
results = cross_val_score(model_Bagging, X, Y, cv=kfold)


In [28]:
print("Bagging Model :",round(results.mean()*100,2))

Bagging Model : 73.0


## **2.Boosting**
AdaBoost Classification

In [30]:
model_Boosting = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)

In [33]:
results = cross_val_score(model_Boosting, X, Y, cv=kfold)

In [35]:
print("Boosting Model :",round(results.mean()*100,2))

Boosting Model : 74.5


## **3.Stacking**
Stacking Ensemble for Classification

In [37]:
# create the sub models
estimators = [] # create empty list for different algorithms

In [38]:
model1 = LogisticRegression(max_iter=500)
estimators.append(('logistic', model1))

model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))

model3 = SVC()
estimators.append(('svc', model3))

In [39]:
# create the ensemble model
ensemble = VotingClassifier(estimators)
results = cross_val_score(ensemble, X, Y, cv=kfold)

In [40]:
print('Stacking Model :',results.mean()*100)

Stacking Model : 79.33333333333333
